In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('data/fin_horo.csv', sep = '#', index_col=0)
df

,date,zodiac,horo,source,len
50,2023-08-07,Овен,Сегодня Овнам стоит провести утро не просто сп...,https://horoscopes.rambler.ru,377
51,2023-08-07,Телец,Сегодня Тельцам стоит лениться до обеда пусть...,https://horoscopes.rambler.ru,366
52,2023-08-07,Близнецы,Сегодня Близнецы думают только о деньгах их н...,https://horoscopes.rambler.ru,374
53,2023-08-07,Рак,Сегодня Ракам стоит провести утро лениво для ...,https://horoscopes.rambler.ru,365
54,2023-08-07,Лев,Сегодня у Львов есть возможность исполнить нес...,https://horoscopes.rambler.ru,354
...,...,...,...,...,...
256199,2023-08-30,Весы,"Следите сегодня за всем, что происходит на раб...",https://astroscope.ru/horoskop/ejednevniy_goro...,453
256200,2023-08-30,Скорпион,Сегодня Скорпионов ждёт приятная встреча. Ныне...,https://astroscope.ru/horoskop/ejednevniy_goro...,451
256202,2023-08-30,Козерог,"Постарайтесь сегодня, дорогие Козероги, быть в...",https://astroscope.ru/horoskop/ejednevniy_goro...,490
256203,2023-08-30,Водолей,День для Водолеев характеризуется неожиданными...,https://astroscope.ru/horoskop/ejednevniy_goro...,429


In [3]:
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [4]:
%pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df[['zodiac', 'horo']], test_size = 0.3,
                                   random_state=42)
train_df = train_df.reset_index(drop= True)
val_df = val_df.reset_index(drop = True)
train_df.shape, val_df.shape

((62118, 2), (26623, 2))

In [6]:
#!g2.1
import torch
device = torch.device ('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
train_df = train_df.rename(columns = {'zodiac': 'source_text', 'horo':'target_text'})
val_df = val_df.rename(columns = {'zodiac': 'source_text', 'horo':'target_text'})

In [8]:
#!g2.1
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/FRED-T5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/FRED-T5-large")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# New Section

In [9]:
#!g2.1
from torch.utils.data import Dataset
class AdditionDataset(Dataset):
    """
        """

    def __init__(self, sentences, targets, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.sentences = sentences
        self.targets = targets

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        sentence = str(self.sentences[index])
        sentence = " ".join(sentence.split())
        target = self.targets[index]

        inputs = self.tokenizer.encode_plus(
            sentence
                                )
        return {"input_ids":self.tokenizer(sentence)['input_ids'], "labels":self.tokenizer(target)['input_ids']}



In [10]:
#!g2.1
train_dataset = AdditionDataset(train_df['source_text'].tolist(),
                                train_df['target_text'].tolist(), tokenizer, 500)
test_dataset = AdditionDataset(val_df['source_text'].tolist(),
                                val_df['target_text'].tolist(), tokenizer, 500)
next(iter(test_dataset))

{'input_ids': [563, 3975, 14945],
 'labels': [6039,
  35947,
  3062,
  1749,
  1747,
  7606,
  1855,
  282,
  11762,
  30406,
  3431,
  304,
  26708,
  225,
  745,
  1747,
  939,
  1269,
  3111,
  16,
  515,
  3716,
  1329,
  322,
  1747,
  2240,
  18,
  3647,
  17708,
  281,
  1714,
  4220,
  4127,
  4220,
  11206,
  895,
  19490,
  3379,
  36220,
  536,
  289,
  4144,
  3328,
  16,
  2391,
  1882,
  1223,
  3379,
  840,
  5848,
  9837,
  1119,
  282,
  19552,
  18]}

In [11]:
#!g2.1
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    evaluation_strategy = 'epoch',
    learning_rate = 2e-4,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    num_train_epochs = 3,
    predict_with_generate = True,
    push_to_hub = False,
    output_dir = "outputs",
    save_steps = 12000

)

In [12]:
#!g2.1
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [13]:
#!g2.1
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(model,
                        args,
                        train_dataset = train_dataset,
                        eval_dataset = test_dataset,
                        data_collator = data_collator,
                        tokenizer = tokenizer,)

trainer.train()

***** Running training *****
  Num examples = 62118
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11649


Epoch,Training Loss,Validation Loss
1,2.817100,2.674166
2,2.676800,2.620472
3,2.607100,2.605619


***** Running Evaluation *****
  Num examples = 26623
  Batch size = 16
***** Running Evaluation *****
  Num examples = 26623
  Batch size = 16
***** Running Evaluation *****
  Num examples = 26623
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=11649, training_loss=2.7398639698275806, metrics={'train_runtime': 3207.8813, 'train_samples_per_second': 58.093, 'train_steps_per_second': 3.631, 'total_flos': 8.942095143786086e+16, 'train_loss': 2.7398639698275806, 'epoch': 3.0})

In [ ]:
#!g2.1
pt_save_directory = "Fred_t5"
tokenizer.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

tokenizer config file saved in Fred_t5/tokenizer_config.json
Special tokens file saved in Fred_t5/special_tokens_map.json
Configuration saved in Fred_t5/config.json
